# USDe Morpho Benchmarks and Reallocation

In this Jupyter notebook, we will automate the process of analyzing and reallocating USDe liquidity within Morpho's financial pools. 

Our primary focus will lie in assessing the current allocation of USDe, the utilization rates, borrowing rates, optimal rates, and capped rates across different liquidity pools distinguished by their Loan-to-Value (LTV) ratios. We aim to ensure that pool allocations are optimized both for the stable functioning of the overall market and for better returns for liquidity providers and borrowers.

1. **Data Analysis**: We will start by performing a data analysis on the current status of various USDe pools. This will include processing important metrics such as total supply, borrow rates, and optimal rates to identify pools that may require attention or reallocation.

2. **Manual Reallocation Criteria**: Based on the analysis, we'll assess the necessity for manual reallocations in accordance with established criteria for active and inactive pools, including factors like minimum balances, maximum utilization, and the safety margin for withdrawing portions of the allocated USDe supply.

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import json
from data_manipulation import *
from api import *

pd.set_option('display.max_columns', None)

## Manual Reallocation Analysis

Defined Pools we look at for allocation and necessary values

In [2]:
pool_keys = [
    'USDe 94.50%',
    'USDe 91.50%',
    'USDe 86%',
    'USDe 77%',
    'sUSDe 94.50%',
    'sUSDe 91.50%',
    'sUSDe 86%',
    'sUSDe 77%',
    'PT-sUSDe-24Oct2024 86%'
]
idle_key = [
    'Idle',
]
idle_col = [
    'LLTV',
    'Total Supply', 
    'Maker Allocation',     
    'Utilization', 
    'Borrow Rate',              
]
columns_ordered = [
    'Status', 
    'LLTV',
    'Total Supply', 
    'Maker Allocation', 
    'Utilization', 
    'Borrow Rate',
    'Supply Cap',
    'Total Borrow', 
    'Maker Borrow', 
    'Optimal Rate', 
    'Capped Borrow Rate', 
    'DSR',
    'Target Borrow Rate',
    'Min Borrow Rate',
    'Max Borrow Rate',
    'Utilization Where Rate Equal to DSR',
    'DSR Adjustment',
    'Total Supply After DSR Adjustment',
    'Maker Supply After DSR Adjustment',
    'Utilization After DSR Adjustment',
    'Inactive Withdrawal',
    'Utilization Where Rate Equal To Min Target',
    'Active Withdrawal',
    'Utilization Where Rate Equal To Max Target',
    'Active Deposits',
    'Manual Adjustment',
    'Total Change',
    'Final Allocation',
    'Final Supply',
    'Final Utilization',
    'Final Borrow Rate',
    'Final Capped Rate',
    'Maker Borrow at Old Utilization',
    'Borrow Rate Change',                   
]

pools_df = pd.DataFrame(index=pool_keys, columns=columns_ordered)
idle_df = pd.DataFrame(index=idle_key, columns=idle_col)
pools_df.head(1)

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
idle_df

,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate
Idle,NaN,NaN,NaN,NaN,NaN


### Retrieve latest values

Connect to BA API and retrieve the data for the pools we are interested in. First define pools by their uuid, to retrieve the data.

In [4]:
market_to_pool = {
    "db760246f6859780f6c1b272d47a8f64710777121118e56e0cdb4b8b744a3094": 'USDe 94.50%',
    "8e6aeb10c401de3279ac79b4b2ea15fc94b7d9cfc098d6c2a1ff7b2b26d9d02c": 'USDe 91.50%',
    "c581c5f70bd1afa283eed57d1418c6432cbff1d862f94eaf58fdd4e46afbb67f": 'USDe 86%',
    "fd8493f09eb6203615221378d89f53fcd92ff4f7d62cca87eece9a2fff59e86f": 'USDe 77%',
    "e475337d11be1db07f7c5a156e511f05d1844308e66e17d2ba5da0839d3b34d9": 'sUSDe 94.50%',
    "1247f1c237eceae0602eab1470a5061a6dd8f734ba88c7cdc5d6109fb0026b28": 'sUSDe 91.50%',
    "39d11026eae1c6ec02aa4c0910778664089cdd97c3fd23f68f7cd05e2e95af48": 'sUSDe 86%',
    "42dcfb38bb98767afb6e38ccf90d59d0d3f0aa216beb3a234f12850323d17536": 'sUSDe 77%',
    "8f46cd82c4c44a090c3d72bd7a84baf4e69ee50331d5deae514f86fe062b0748": 'PT-sUSDe-24Oct2024 86%'
}

In [5]:
# market_to_pool.keys()

Retrieve the data from API

In [6]:
# Base URL for market data
base_market_url = "https://morpho-api.blockanalitica.com/markets/{}/historic/?days_ago=0&network=ethereum"
# Base URL for wallet data
base_wallet_url = "https://morpho-api.blockanalitica.com/markets/{}/wallets/?network=ethereum&order=-supply&p_size=4&type=suppliers"
# URL for vault data
vaults_url = "https://morpho-api.blockanalitica.com/vaults/0x73e65dbd630f90604062f6e02fab9138e713edd9/pools/?days_ago=0&network=ethereum&order=-supply_usd&p=1&p_size=15"
# URL for DSR rate
dsr_url = "https://info-sky.blockanalitica.com/save/"
# The wallet address we are interested in
target_wallet = "0x73e65dbd630f90604062f6e02fab9138e713edd9"

# Assuming market_to_pool is defined
api_data = ApiBA(base_market_url, base_wallet_url, market_to_pool.keys(), target_wallet, vaults_url, dsr_url)

With data from API populate the dataframe

In [7]:
market_data = PoolDataHandler(pools_df, market_to_pool)
pools_df = market_data.populate_dataframe(api_data.fetch_data())
pools_df


Updated DataFrame:


,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,0.945,888775,712515,0.8984,0.1082,10000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 91.50%,NaN,0.915,38211992,38211992,0.9024,0.0794,200000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 86%,NaN,0.86,70250992,70175787,0.9,0.0809,500000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 77%,NaN,0.77,1605088,1605088,0.8958,0.0693,1000000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 94.50%,NaN,0.945,540854,305807,0.9301,0.3074,10000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 91.50%,NaN,0.915,34163260,34163260,0.9019,0.0616,200000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 86%,NaN,0.86,78885760,78832929,0.8589,0.0677,500000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 77%,NaN,0.77,100048,74507,0.7405,0.0102,1000000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-24Oct2024 86%,NaN,0.86,100390821,100390820,0.9122,0.0948,100000000,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#index_to_drop = 'sUSDe 77%PT-sUSDe-24Oct2024 86%'
#pools_df = pools_df.drop(index_to_drop)

To see the allocation in the Idle matket:

In [10]:
idle_market_to_pool = {"57f4e42c0707d3ae0ae39c9343dcba78ff79fa663da040eca45717a9b0b0557f": 'Idle'}
idle_api_data = ApiBA(base_market_url, base_wallet_url, idle_market_to_pool.keys(), target_wallet, vaults_url, dsr_url)
idle_data = PoolDataHandler(idle_df, idle_market_to_pool)
idle_df = idle_data.populate_dataframe(idle_api_data.fetch_data())
idle_df


Updated DataFrame:


,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,DSR
Idle,0.0,25590706,25590706,0.0,0.0,24519928653854223487948685977518604288,0.06


### Reallocation Metaparameters

They are set as done by Monet. Change only in case you know what you are doing!

In [11]:
realloc_metaparm = {
    'active_pool' : {
        'min_balance' : 10000000,
        'max_utilization' : 0.9,
        'max_portion_to_withdraw' : 0.1,
        'allocation_significance_threshold' : 10000,
    },
    'inactive_pool' : {
        'min_balance' : 100000,
        'max_utilization' : 0.93,
        'max_portion_to_withdraw' : 1,
        'allocation_significance_threshold' : 10000,
    },
}

In [12]:
pools_analysis = PoolAnalysis(pools_df, realloc_metaparm)

#### Classify market as Active or Inactive

Change the status of the pool you want to change by changing the pool status to `Active` or `Inactive`

In [13]:
pools_df.loc['USDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['USDe 91.50%', 'Status'] = 'Active'
pools_df.loc['USDe 86%', 'Status'] = 'Active'
pools_df.loc['USDe 77%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 91.50%', 'Status'] = 'Active'
pools_df.loc['sUSDe 86%', 'Status'] = 'Active'
pools_df.loc['sUSDe 77%', 'Status'] = 'Inactive'
pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Status'] = 'Active'

In [ ]:
#pools_analysis.define_active_or_inactive()

### Manual Adjustment

If you want to add a manual adjustment, do it here by uncommenting the line of code with pool you want to adjust and change the number inside the parenthesis.

In [ ]:
#pools_df.loc['USDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Manual Adjustment'] = int(0)

### Reallocation Dataframe

Get the final table with all data. For easier overview focus on the following tables

In [14]:
pools_analysis = PoolAnalysis(pools_df, realloc_metaparm)

In [15]:
pools_analysis.update_pool_dataframe()

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,DSR,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to DSR,DSR Adjustment,Total Supply After DSR Adjustment,Maker Supply After DSR Adjustment,Utilization After DSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,Inactive,0.945,888775,712515,0.8984,0.1082,10000000,798475,640123,0.1083,0.1082,0.06,0.1324,0.1059,0.1456,0.3648,0,888775,712515,0.8984,0,0.8734,0,0.9115,0,0,0,712515,888775,0.8984,0.1082,0.1082,640123,0.0
USDe 91.50%,Active,0.915,38211992,38211992,0.9024,0.0794,200000000,34482501,34482501,0.0741,0.0741,0.06,0.1324,0.1059,0.1456,0.6717,0,38211992,38211992,0.9024,0,0.9143,0,0.9322,0,0,0,38211992,38211992,0.9024,0.0794,0.0741,34482501,0.0
USDe 86%,Active,0.86,70250992,70175787,0.9,0.0809,500000000,63225892,63158207,0.0809,0.0809,0.06,0.1324,0.1059,0.1456,0.59,0,70250992,70175787,0.9,0,0.9103,0,0.9267,0,0,0,70175787,70250992,0.9,0.0809,0.0809,63158208,0.0
USDe 77%,Inactive,0.77,1605088,1605088,0.8958,0.0693,1000000000,1437837,1437837,0.0695,0.0693,0.06,0.1324,0.1059,0.1456,0.736,0,1605088,1605088,0.8958,0,0.9175,0,0.9365,0,0,0,1605088,1605088,0.8958,0.0693,0.0693,1437837,0.0
sUSDe 94.50%,Inactive,0.945,540854,305807,0.9301,0.3074,10000000,503048,284430,0.1615,0.1615,0.06,0.1324,0.1059,0.1456,0.1458,0,540854,305807,0.9301,0,0.4869,0,0.7819,0,0,0,305807,540854,0.9301,0.3073,0.1615,284431,-0.0001
sUSDe 91.50%,Active,0.915,34163260,34163260,0.9019,0.0616,200000000,30811844,30811844,0.0583,0.0583,0.06,0.1324,0.1059,0.1456,0.901,0,34163260,34163260,0.9019,0,0.9272,0,0.9499,0,0,0,34163260,34163260,0.9019,0.0616,0.0583,30811844,0.0
sUSDe 86%,Active,0.86,78885760,78832929,0.8589,0.0677,500000000,67754979,67709602,0.0701,0.0677,0.06,0.1324,0.1059,0.1456,0.7271,0,78885760,78832929,0.8589,0,0.917,-3602450,0.9359,0,0,-3602450,75230479,75283310,0.9,0.0701,0.0701,64615458,0.0024
sUSDe 77%,Inactive,0.77,100048,74507,0.7405,0.0102,1000000000,74085,55172,0.0118,0.0102,0.06,0.1324,0.1059,0.1456,1,-48,100000,74459,0.7409,0,1,0,1,0,0,0,74507,100048,0.7405,0.0102,0.0102,55172,0.0
PT-sUSDe-24Oct2024 86%,Active,0.86,100390821,100390820,0.9122,0.0948,100000000,91576506,91576505,0.0694,0.0694,0.06,0.1324,0.1059,0.1456,0.7375,0,100390821,100390820,0.9122,0,0.9175,0,0.9366,0,0,0,100390820,100390821,0.9122,0.0948,0.0694,91576506,0.0


Save the data if you want to perform later analysis or visualizations on this

In [ ]:
#pools_analysis.pool_df.to_json(f'reallocations_{datetime.datetime.now().strftime('%Y_%m_%d')}.json')

### Average Utilization, Net and Running Change

Some basic statistics for reallocation

In [16]:
average_utilization = pools_analysis.pool_df['Utilization'].mean()
print(f'The average utilization of the pools is: {average_utilization * 100:.2f}%')

The average utilization of the pools is: 88.22%


The net changes based on the type of adjusment are as follows

In [17]:
net_change_df = pools_analysis.pool_df[['DSR Adjustment', 'Inactive Withdrawal', 'Active Withdrawal', 'Active Deposits', 'Manual Adjustment']].agg('sum').to_frame().T
net_change_df.columns = ['DSR Adjustment Sum', 'Inactive Withdrawal Sum', 'Active Withdrawal Sum', 'Active Deposits Sum', 'Manual Adjustment Sum']
net_change_df = net_change_df.round(0).astype(int)

net_change_df

,DSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-48,0,-3602450,0,0


The running total of changes by the type of adjustment are as follows, which will result in new adjusted Idle supply

In [18]:
cumulative_sums = net_change_df.cumsum(axis=1).iloc[-1]
cumulative_sums_df = pd.DataFrame({
    'DSR Adjustment Sum': [cumulative_sums['DSR Adjustment Sum']],
    'Inactive Withdrawal Sum': [cumulative_sums['Inactive Withdrawal Sum']],
    'Active Withdrawal Sum': [cumulative_sums['Active Withdrawal Sum']],
    'Active Deposits Sum': [cumulative_sums['Active Deposits Sum']],
    'Manual Adjustment Sum': [cumulative_sums['Manual Adjustment Sum']]
})
cumulative_sums_df = cumulative_sums_df.round(0).astype(int)

cumulative_sums_df

,DSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-48,-48,-3602498,-3602498,-3602498


## Manual Reallocation Dataframe

The following table shows the change and new allocated supply with wei allocation

In [19]:
new_pool_data = {
    'Net Change': pools_df['Total Change'],
    'New Allocation Supply': pools_df['Final Allocation']
}
manual_realloaction_pools = pd.DataFrame(new_pool_data)
manual_realloaction_pools['New Allocation Wei'] = manual_realloaction_pools['New Allocation Supply'] * pow(10, 18)

In [20]:
# Create a styled DataFrame
styled_df = manual_realloaction_pools.style.format({
    'Net Change': '{:.0f}',
    'New Allocation Supply': '{:.0f}',
    'New Allocation Wei': '{:.0f}'
}).map(PoolAnalysis.color_net_change, subset=['Net Change'])

styled_df

,Net Change,New Allocation Supply,New Allocation Wei
USDe 94.50%,0,712515,712514999999999946784768
USDe 91.50%,0,38211992,38211991999999999876268032
USDe 86%,0,70175787,70175786999999996547301376
USDe 77%,0,1605088,1605088000000000075497472
sUSDe 94.50%,0,305807,305806999999999985582080
sUSDe 91.50%,0,34163260,34163259999999999405457408
sUSDe 86%,-3602450,75230479,75230478999999998341414912
sUSDe 77%,0,74507,74507000000000003407872
PT-sUSDe-24Oct2024 86%,0,100390820,100390819999999999059427328


In [21]:
print(f'The total net change in allocation is: ${manual_realloaction_pools["Net Change"].sum():.0f}')

The total net change in allocation is: $-3602450


## Pool Overview Dataframe

A brief statistics for pool overview

In [22]:
pool_stats = [
    'Total Non-Idle Allocation',
    'Supply Weighted LLTV',
    'Supply Weighted sUSDe',
    'Average Borrow Rate',
    'Average Capped Rate',
    'Rate at Prior Equilibrium',
]
columns_stats = [
    'Current',
    'Future',
    'Change'                 
]

pool_overview = pd.DataFrame(index=pool_stats, columns=columns_stats)
statistics = PoolOverview(pools_df, pool_overview)

In [23]:
statistics.update_pool_overview()

,Current,Future,Change
Total Non-Idle Allocation,324472705,320870255,-0.011102
Supply Weighted LLTV,0.872069,0.872204,0.000155
Supply Weighted sUSDe,0.349418,0.342114,-0.020904
Average Borrow Rate,0.08019,0.08075,0.006983
Average Capped Rate,0.07105,0.071611,0.007882
Rate at Prior Equilibrium,0.08019,0.08075,0.006983


In [24]:
pool_styled_df = pool_overview.style.format({
    'Current': '{:.3f}',
    'Future': '{:.3f}',
    'Change': '{:.2f}%'
}).map(PoolAnalysis.color_net_change, subset=['Change'])

pool_styled_df

,Current,Future,Change
Total Non-Idle Allocation,324472705.000,320870255.000,-0.01%
Supply Weighted LLTV,0.872,0.872,0.00%
Supply Weighted sUSDe,0.349,0.342,-0.02%
Average Borrow Rate,0.080,0.081,0.01%
Average Capped Rate,0.071,0.072,0.01%
Rate at Prior Equilibrium,0.080,0.081,0.01%


### Current Debt Ceiling

In [25]:
current_ddm_target_debt = int(pool_overview.at['Total Non-Idle Allocation', 'Current'] + idle_df.at['Idle', 'Total Supply'])

print(f'The current DDM target debt is: ${current_ddm_target_debt}')

future_ddm_target_debt = math.ceil((int(pool_overview.at['Total Non-Idle Allocation', 'Future'])+ 5000000)/ 25000000) * 25000000

print(f'The future DDM target debt is: ${future_ddm_target_debt}')

The current DDM target debt is: $350063411
The future DDM target debt is: $350000000


## JSON Logic

Include only pools that change in allocation

### Allocation to USDe

[
MakerDAO: Dai Token (DAI),
Ethena: USDe (USDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E18)
],
Allocation WEI

### Allocation to sUSDe

[
MakerDAO: Dai Token (DAI),
Ethena: Staked USDe (sUSDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E18)
]
Allocation WEI

### Everything else goes to the idle market

[
MakerDAO: Dai Token (DAI),
Burn Address,
Burn Address,
Burn Address,
0
]
Uint256 Maximum Value

In [27]:
manual_reallocation_pools = manual_realloaction_pools

manual_reallocation_pools.index = [
    'USDe 94.50%', 'USDe 91.50%', 'USDe 86%', 'USDe 77%', 
    'sUSDe 94.50%', 'sUSDe 91.50%', 'sUSDe 86%', 'sUSDe 77%', 
    'PT-sUSDe-24Oct2024 86%'
]

# Define the base address values
base_addresses = {
    "USDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3", # USDe
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "sUSDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x9D39A5DE30e57443BfF2A8307A4256c8797A3497", # sUSDe
        "0x5D916980D5Ae1737a8330Bf24dF812b2911Aae25", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "PT-sUSDe-24Oct2024": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0xAE5099C39f023C91d3dd55244CAFB36225B0850E", # PT-sUSDe-24Oct2024
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "Idle": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x0000000000000000000000000000000000000000", # Idle
        "0x0000000000000000000000000000000000000000", # MorphoChainlinkOracleV2
        "0x0000000000000000000000000000000000000000" # AdaptiveCurveIRM
    ]
}

# Function to determine the pool type based on the allocation type (in the index)
def determine_pool_type(allocation_type):
    if "USDe" in allocation_type:
        return base_addresses["USDe"]
    elif "sUSDe" in allocation_type:
        return base_addresses["sUSDe"]
    elif "PT-sUSDe-24Oct2024" in allocation_type:
        return base_addresses["PT-sUSDe-24Oct2024"]
    else:
        return base_addresses["Idle"]

# Initialize the JSON structure
output_json = []

# Iterate over the DataFrame rows, but only process rows where Net Change is not 0
for allocation_type, row in manual_reallocation_pools.iterrows():
    if row['Net Change'] != 0:
        pool_type = determine_pool_type(allocation_type)
        lltv = str(int(float(allocation_type.split()[1].replace('%', '')) * 10**18))  # Convert percentage to LLTV
        pool = pool_type + [lltv]

        allocation_wei = row['New Allocation Wei']
        
        output_json.append([pool, allocation_wei])

# Add the 'Idle' pool entry
idle_pool = base_addresses["Idle"] + ["0"]
idle_allocation_wei = "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff"
output_json.append([idle_pool, idle_allocation_wei])

# Convert the list to JSON formatted string
output_str = json.dumps(output_json, indent=2)

# Print or save the JSON formatted string
print(output_str)

# Optionally, save to a file
with open("morpho_vault_realloc.json", "w") as file:
    file.write(output_str)

[
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3",
      "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35",
      "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC",
      "86000000000000000000"
    ],
    75230479000000000000000000
  ],
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x0000000000000000000000000000000000000000",
      "0x0000000000000000000000000000000000000000",
      "0x0000000000000000000000000000000000000000",
      "0"
    ],
    "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff"
  ]
]
